In [1]:
import os
import sys
import glob
from os import write

from pyparsing.diagram import template
from tqdm import tqdm
import pandas as pd
import numpy as np
import ast
import scipy.stats as st

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.iDrink.iDrinkValPlots import ignore_id_p, murphy_measures

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities

In [2]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_stat_cat = os.path.join(root_stat, '02_categorical')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots_cont = os.path.join(dir_stat_cont, '02_plots')
dir_plots_cat = os.path.join(dir_stat_cat, '02_plots')

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')
df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']  # Becaus bad calibration reprojection error

murphy_measures = ['PeakVelocity_mms', 'elbowVelocity', 'tTopeakV_s', 'tToFirstpeakV_s',
       'tTopeakV_rel', 'tToFirstpeakV_rel', 'NumberMovementUnits',
       'InterjointCoordination', 'trunkDisplacementMM', 'trunkDisplacementDEG',
       'ShoulderFlexionReaching', 'ElbowExtension', 'shoulderAbduction',
       'shoulderFlexionDrinking']

In [3]:
def get_title_measure_name(measure):
    """returns a string based on the murphy measure for a figure_title"""
    match measure:
        case 'PeakVelocity_mms':
            title = 'Peak Endeffector Velocity'
        case 'elbowVelocity':
            title = 'Peak Elbow Velocity'
        case 'tTopeakV_s':
            title = 'Time to Peak Velocity'
        case 'tToFirstpeakV_s':
            title = 'Time to First Peak Velocity'
        case 'tTopeakV_rel':
            title = 'Relative time to Peak Velocity relative'
        case 'tToFirstpeakV_rel':
            title = 'Relative time to First Peak Velocity'
        case 'NumberMovementUnits':
            title = 'Number of Movement Units'
        case 'InterjointCoordination':
            title = 'Interjoint Coordination'
        case 'trunkDisplacementMM':
            title = 'Trunk Displacement'
        case 'trunkDisplacementDEG':
            title = 'Trunk Displacement'
        case 'ShoulderFlexionReaching':
            title = 'Shoulder Flexion Reaching'
        case 'ElbowExtension':
            title = 'Elbow Extension'
        case 'shoulderAbduction':
            title = 'Shoulder Abduction'
        case 'shoulderFlexionDrinking':
            title = 'Shoulder Flexion Drinking'
            title = 'Trunk Angle'
        case 'hand_vel':
            title = 'Hand Velocity'
        case 'elbow_vel':
            title = 'Elbow Velocity'
        case 'trunk_disp':
            title = 'Trunk Displacement'
        case 'trunk_ang':
            title = 'Trunk Angle'
        case 'elbow_flex_pos':
            title = 'Elbow Flexion'
        case 'shoulder_flex_pos':
            title = 'Shoulder Flexion'
        case 'shoulder_abduction_pos':
            title = 'Shoulder Abduction'
        case _:
            title = ''
    return title

def get_unit(kin):

    cases_deg = ['trunk_ang', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos',
                 'trunkDisplacementDEG', 'ShoulderFlexionReaching', 'ElbowExtension',
                 'shoulderAbduction', 'shoulderFlexionDrinking']

    match kin:
        case 'hand_vel' | 'PeakVelocity_mms':
            unit = 'mm/s'
        case 'elbow_vel' | 'elbowVelocity':
            unit = 'deg/s'
        case 'trunk_disp' | 'trunkDisplacementMM':
            unit = 'mm'
        case k if k in cases_deg:
            unit = 'deg'
        case 'tTopeakV_s' | 'tToFirstpeakV_s' :
            unit = 's'
        case 'tTopeakV_rel' | 'tToFirstpeakV_rel':
            unit = '%'
        case _:
            unit = ''

    return unit

def get_cad(df, measure):
    match measure:
        case 'PeakVelocity_mms' | 'hand_vel':
            measure_name = 'peak_V'
        case 'elbowVelocity' | 'elbow_vel':
            measure_name = 'peak_V_elb'
        case 'tTopeakV_s':
            measure_name = 't_to_PV'
        case 'tToFirstpeakV_s':
            measure_name = 't_first_PV'
        case 'tTopeakV_rel':
            measure_name = 't_PV_rel'
        case 'tToFirstpeakV_rel':
            measure_name = 't_first_PV_rel'
        case 'NumberMovementUnits':
            measure_name = 'n_mov_units'
        case 'InterjointCoordination':
            measure_name = 'interj_coord'
        case 'trunkDisplacementMM' | 'trunk_disp':
            measure_name = 'trunk_disp'
        case 'trunkDisplacementDEG' | 'trunk_ang':
            return None
        case 'ShoulderFlexionReaching' | 'elbow_flex_pos':
            measure_name = 'arm_flex_reach'
        case 'ElbowExtension' | 'shoulder_flex_pos':
            measure_name = 'elb_ext'
        case 'shoulderAbduction' | 'shoulder_abduction_pos':
            measure_name = 'arm_abd'
        case 'shoulderFlexionDrinking' | 'shoulder_flex_pos':
            measure_name = 'arm_flex_drink'
        case _:
            return

    return df.loc[0, measure_name]

# Bland Altman Tables
## Kinematic Measures

In [4]:
df_murphy = df_murphy[df_murphy['id_p'].isin(ignore_id_p) == False]
df_murphy.drop(columns=['valid', 'ReachingStart',
       'ForwardStart', 'DrinkingStart', 'BackStart', 'ReturningStart',
       'RestStart', 'TotalMovementTime'], inplace=True)

In [5]:
df_murphy.columns

Index(['identifier', 'id_s', 'id_p', 'id_t', 'side', 'condition',
       'PeakVelocity_mms', 'elbowVelocity', 'tTopeakV_s', 'tToFirstpeakV_s',
       'tTopeakV_rel', 'tToFirstpeakV_rel', 'NumberMovementUnits',
       'InterjointCoordination', 'trunkDisplacementMM', 'trunkDisplacementDEG',
       'ShoulderFlexionReaching', 'ElbowExtension', 'shoulderAbduction',
       'shoulderFlexionDrinking'],
      dtype='object')

In [6]:
id_s = 'S001'
id_p = 'P07'
measure = 'PeakVelocity_mms'

In [7]:
df_altman = pd.DataFrame(columns=['id_s', 'id_p', 'measure', 'mean_err', 'lower_loa', 'upper_loa'])

In [51]:
df_murphy_omc_s = df_murphy[df_murphy['id_s'] == 'S15133']
df_murphy_mmc = df_murphy[df_murphy['id_s'] != 'S15133']


idx_s = df_murphy_mmc['id_s'].unique()
idx_p = set(df_murphy_omc_s['id_p'].unique()).intersection(set(df_murphy_mmc_s['id_p'].unique()))

In [124]:
df_murphy_mmc

,identifier,id_s,id_p,id_t,side,condition,PeakVelocity_mms,elbowVelocity,tTopeakV_s,tToFirstpeakV_s,tTopeakV_rel,tToFirstpeakV_rel,NumberMovementUnits,InterjointCoordination,trunkDisplacementMM,trunkDisplacementDEG,ShoulderFlexionReaching,ElbowExtension,shoulderAbduction,shoulderFlexionDrinking
827,S001_P07_T010,S001,P07,T010,L,unaffected,988.3890,3.1275,5.300,5.300,70.572570,70.572570,4,-0.0645,48099.5118,11.7158,56.2339,38.0811,27.3560,57.9720
828,S001_P07_T011,S001,P07,T011,L,unaffected,828.4310,2.9747,4.500,4.500,57.397959,57.397959,7,-0.1680,47273.5844,11.7749,57.8504,34.6043,28.4776,59.3558
829,S001_P07_T012,S001,P07,T012,L,unaffected,1004.9400,3.2473,3.733,3.733,58.237129,58.237129,4,-0.1204,38712.0994,9.5909,58.5359,34.7820,28.2210,57.7717
830,S001_P07_T013,S001,P07,T013,L,unaffected,1035.6825,2.9733,5.183,5.183,77.242921,77.242921,6,0.0151,46388.7392,11.3752,57.8601,37.7356,28.6471,59.5379
831,S001_P07_T014,S001,P07,T014,L,unaffected,786.6286,3.2573,5.617,5.617,71.011378,71.011378,8,-0.0481,53873.3402,13.0564,56.0133,36.0599,28.2610,56.9061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9903,S022_P252_T079,S022,P252,T079,L,affected,971.3752,2.6609,5.017,5.017,65.240572,65.240572,8,-0.5947,51752.7295,7.2416,63.0425,42.1960,10.2865,42.2069
9904,S022_P252_T080,S022,P252,T080,L,affected,824.9142,2.4072,5.600,5.600,72.072072,72.072072,11,-0.5754,60893.9025,7.4621,62.4105,42.5817,11.3960,43.9927
9905,S022_P252_T081,S022,P252,T081,L,affected,932.6968,2.7196,5.283,5.283,69.149215,69.149215,7,-0.4957,48738.4177,6.7521,62.8186,42.8004,10.4943,45.6586
9906,S022_P252_T082,S022,P252,T082,L,affected,949.9188,2.6934,5.300,5.300,68.298969,68.298969,11,-0.5459,43226.6575,6.5964,64.0824,39.0516,12.2374,41.2458


In [15]:
idx_s = ['S001']
#idx_p = ['P07']

In [55]:
df_altman = pd.DataFrame(columns=['id_s', 'id_p', 'measure', 'mean_err', 'lower_loa', 'upper_loa'])
total = len(idx_s) * len(murphy_measures)

progbar = tqdm(total=total, desc='Processing')
for measure in murphy_measures:
    
    measure_name = get_title_measure_name(measure)
    unit = get_unit(measure)
    colors = px.colors.qualitative.Plotly
    
    for id_s in idx_s:
        
        progbar.set_description(f'Processing {id_s} \t {measure}')
        
        df_murphy_mmc_s = df_murphy[df_murphy['id_s'] == id_s]
        i = 0
        
        fig_val_val = go.Figure()
        max_val = 0
        
        df_omc_fit = None
        df_mmc_fit = None
        
        
        for id_p in idx_p:
            df_murphy_omc_p = df_murphy_omc_s[df_murphy_omc_s['id_p'] == id_p]
            df_murphy_mmc_p = df_murphy_mmc_s[df_murphy_mmc_s['id_p'] == id_p]
            
            idx_t = set(df_murphy_omc_p['id_t'].unique()).intersection(set(df_murphy_mmc_p['id_t'].unique()))
        
            df_murphy_omc_p = df_murphy_omc_p[df_murphy_omc_p['id_t'].isin(idx_t)].sort_values(by='id_t')
            df_murphy_mmc_p = df_murphy_mmc_p[df_murphy_mmc_p['id_t'].isin(idx_t)].sort_values(by='id_t')
            
            if df_omc_fit is None:
                df_omc_fit = df_murphy_omc_p
                df_mmc_fit = df_murphy_mmc_p
            else:
                df_omc_fit = pd.concat([df_omc_fit, df_murphy_omc_p], ignore_index=True)
                df_mmc_fit = pd.concat([df_mmc_fit, df_murphy_mmc_p], ignore_index=True)
            
            # get values and add to DataFrame
            means = np.nanmean([df_murphy_omc_p[measure].values, df_murphy_mmc_p[measure].values], axis=0)
            
            diff = df_murphy_mmc_p[measure].values - df_murphy_omc_p[measure].values
            mean_err = np.nanmean(diff)
            std = np.nanstd(diff, ddof=1)
            unit = get_unit(measure)
            
            # Confidence level
            confidence = 0.95
            loas = st.norm.interval(confidence, mean_err, std)
            lower_loa = loas[0]
            upper_loa = loas[1]
            
            df_altman = pd.concat([df_altman, pd.DataFrame({'id_s': id_s, 'id_p': id_p, 'measure': measure, 'mean_err': mean_err, 'lower_loa': lower_loa, 'upper_loa': upper_loa}, index = [0])], ignore_index=True)
            
            # Make Value-Value Plot
            try:
                new_max = max(np.nanmax(df_murphy_mmc_p[measure].values), np.nanmax(df_murphy_omc_p[measure].values))
                if max_val < new_max:
                    max_val = new_max
            except:
                pass
            
            fig_val_val.add_trace(go.Scatter(x=df_murphy_mmc_p[measure].values, y=df_murphy_omc_p[measure].values, mode='markers', name=f'{id_p}', marker=dict(color=colors[i], size=10)))
            fig_val_val.update_layout(title=f'{measure_name} for {id_s}', xaxis_title=f'MMC {measure_name} [{unit}]', yaxis_title=f'OMC {measure_name} [{unit}]', template='plotly')
            i+=1
            
        fig_val_val.add_trace(go.Scatter(x=[0, max_val], y=[0, max_val], mode='lines', name='Line of Equality', line=dict(color='grey', width=2, dash='dash')))
        
        try:
            regress = st.linregress(x=df_mmc_fit[measure].values, y=df_omc_fit[measure].values)
            fig_val_val.add_trace(go.Scatter(x=df_mmc_fit[measure].values, y=regress.intercept + regress.slope * df_mmc_fit[measure].values, mode='lines', name='Regression Line', line=dict(color='red', width=2)))
            
            fig_val_val.update_layout(template='plotly', height = 1000, width = 1000)
        
            fig_val_val.add_annotation(
                x=max_val,  # Set the annotation near the maximum x value
                y=0,  # Set the annotation near the minimum y value
                text=f"Slope: {regress.slope:.2f}",  # Display the slope with 2 decimal places
                showarrow=False,  # No arrow needed
                font=dict(size=12),  # Adjust font size
                xanchor="right",  # Align to the right
                yanchor="bottom"  # Align to the bottom
                )
        except:
            pass
        

        
        # If nregress is None, do not write the image
        
        progbar.update(1)
progbar.close()
            
            
            
            
            
            
            
            
            
            
            
            
            


Processing S023 	 ElbowExtension:  82%|████████▏ | 299/364 [00:47<00:11,  5.88it/s]C:\Users\johan\AppData\Local\Temp\ipykernel_37316\894080207.py:45: RuntimeWarning:

Mean of empty slice

C:\Users\johan\anaconda3\envs\iDrink_GUI\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning:

Degrees of freedom <= 0 for slice.

C:\Users\johan\AppData\Local\Temp\ipykernel_37316\894080207.py:45: RuntimeWarning:

Mean of empty slice

C:\Users\johan\anaconda3\envs\iDrink_GUI\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning:

Degrees of freedom <= 0 for slice.

C:\Users\johan\AppData\Local\Temp\ipykernel_37316\894080207.py:45: RuntimeWarning:

Mean of empty slice

C:\Users\johan\anaconda3\envs\iDrink_GUI\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning:

Degrees of freedom <= 0 for slice.

C:\Users\johan\AppData\Local\Temp\ipykernel_37316\894080207.py:45: RuntimeWarning:

Mean of empty slice

C:\Users\johan\anaconda3\envs\iDrink_GUI\Lib\site-packages\numpy

In [47]:
df_altman


,id_s,id_p,measure,mean_err,lower_loa,upper_loa
0,S001,P07,PeakVelocity_mms,-69.611863,-490.762128,351.538402
1,S001,P242,PeakVelocity_mms,85.617876,-13.267361,184.503113
2,S001,P10,PeakVelocity_mms,398.825990,-5554.001961,6351.653941
3,S001,P241,PeakVelocity_mms,-8.117272,-182.269499,166.034955
4,S001,P08,PeakVelocity_mms,-29.342493,-124.009901,65.324915
...,...,...,...,...,...,...
121,S001,P08,shoulderFlexionDrinking,-12.751107,-14.933852,-10.568361
122,S001,P12,shoulderFlexionDrinking,-11.787162,-28.736926,5.162603
123,S001,P15,shoulderFlexionDrinking,-10.229149,-70.708376,50.250078
124,S001,P252,shoulderFlexionDrinking,-12.581788,-15.929675,-9.233900


In [48]:
df_latex = df_altman.groupby(['id_s', 'measure']).mean(numeric_only=True)

In [35]:
df_altman.groupby(['id_s', 'measure']).mean(numeric_only=True)

,,mean_err
id_s,measure,
S001,PeakVelocity_mms,1856.080354


In [49]:
df_latex

mean_err     lower_loa     upper_loa
id_s measure                                                          
S001 ElbowExtension              11.294520     -1.896861     24.485901
     InterjointCoordination       0.082984     -0.221662      0.387630
     NumberMovementUnits          0.595074     -5.510456      6.700604
     PeakVelocity_mms          1856.080354   -852.635829   4564.796537
     ShoulderFlexionReaching    -10.812213    -20.520365     -1.104061
     elbowVelocity                1.503489     -1.136684      4.143661
     shoulderAbduction           16.816401      8.705198     24.927605
     shoulderFlexionDrinking    -10.390070    -23.920934      3.140794
     tToFirstpeakV_rel            0.135468    -39.441083     39.712018
     tToFirstpeakV_s             -0.003458     -3.000913      2.993997
     tTopeakV_rel                 0.135468    -39.441083     39.712018
     tTopeakV_s                  -0.003458     -3.000913      2.993997
     trunkDisplacementDEG         2.707858    -10.881653     16.297370
     trunkDisplacementMM      17150.596933 -48382.753209  82683.947074

In [50]:
print(df_latex.style.to_latex())

\begin{tabular}{llrrr}
 &  & mean_err & lower_loa & upper_loa \\
id_s & measure &  &  &  \\
\multirow[c]{14}{*}{S001} & ElbowExtension & 11.294520 & -1.896861 & 24.485901 \\
 & InterjointCoordination & 0.082984 & -0.221662 & 0.387630 \\
 & NumberMovementUnits & 0.595074 & -5.510456 & 6.700604 \\
 & PeakVelocity_mms & 1856.080354 & -852.635829 & 4564.796537 \\
 & ShoulderFlexionReaching & -10.812213 & -20.520365 & -1.104061 \\
 & elbowVelocity & 1.503489 & -1.136684 & 4.143661 \\
 & shoulderAbduction & 16.816401 & 8.705198 & 24.927605 \\
 & shoulderFlexionDrinking & -10.390070 & -23.920934 & 3.140794 \\
 & tToFirstpeakV_rel & 0.135468 & -39.441083 & 39.712018 \\
 & tToFirstpeakV_s & -0.003458 & -3.000913 & 2.993997 \\
 & tTopeakV_rel & 0.135468 & -39.441083 & 39.712018 \\
 & tTopeakV_s & -0.003458 & -3.000913 & 2.993997 \\
 & trunkDisplacementDEG & 2.707858 & -10.881653 & 16.297370 \\
 & trunkDisplacementMM & 17150.596933 & -48382.753209 & 82683.947074 \\
\end{tabular}



In [45]:
print(df_latex.to_markdown())

|                                     |       mean_err |
|:------------------------------------|---------------:|
| ('S001', 'ElbowExtension')          |    11.2945     |
| ('S001', 'InterjointCoordination')  |     0.0829839  |
| ('S001', 'NumberMovementUnits')     |     0.595074   |
| ('S001', 'PeakVelocity_mms')        |  1856.08       |
| ('S001', 'ShoulderFlexionReaching') |   -10.8122     |
| ('S001', 'elbowVelocity')           |     1.50349    |
| ('S001', 'shoulderAbduction')       |    16.8164     |
| ('S001', 'shoulderFlexionDrinking') |   -10.3901     |
| ('S001', 'tToFirstpeakV_rel')       |     0.135468   |
| ('S001', 'tToFirstpeakV_s')         |    -0.00345812 |
| ('S001', 'tTopeakV_rel')            |     0.135468   |
| ('S001', 'tTopeakV_s')              |    -0.00345812 |
| ('S001', 'trunkDisplacementDEG')    |     2.70786    |
| ('S001', 'trunkDisplacementMM')     | 17150.6        |
